In [ ]:
import random

import matplotlib.pyplot as plt
import polars as pl

Read dataframe


In [ ]:
filename = "filedb.feather"

In [ ]:
df = pl.read_ipc(filename)

In [ ]:
# shuffle dataframe
def shuffle(df: pl.DataFrame, rand_col="rnd") -> pl.DataFrame:
    return df.with_columns(pl.Series(rand_col, [random.random() for _ in range(len(df))])).sort(rand_col).drop(rand_col)

In [ ]:
# save shuffled
shuffle(df).write_ipc(filename)

plot resolution vs modified time

In [ ]:
plt.figure(figsize=(5, 10), dpi=300)
plt.scatter(
    x=df.select("modifiedtime"),
    y=df.with_columns(sum_res=pl.col("resolution").apply(lambda lst: sum(lst))).select("sum_res"),
    s=0.05,
    c=df.select("checkedtime"),
    alpha=0.5
)
plt.xlabel("modifiedtime")
plt.ylabel("sum resolution")